In [26]:
%%capture
!pip install -U -q langgraph langchain-community langchain-anthropic openai langchain-openai

In [2]:
import os

os.environ["ANTHROPIC_API_KEY"] = "nnn"
os.environ['OPENAI_API_KEY'] = "nnn"
os.environ["TAVILY_API_KEY"] = ""

In [3]:
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda

from langgraph.prebuilt import ToolNode

from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph.message import AnyMessage, add_messages


class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
def handle_tool_error(state) -> dict:
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls
    return {
        "messages": [
            ToolMessage(
                content=f"Error: {repr(error)}\n please fix your mistakes.",
                tool_call_id=tc["id"],
            )
            for tc in tool_calls
        ]
    }


def create_tool_node_with_fallback(tools: list) -> dict:
    return ToolNode(tools).with_fallbacks(
        [RunnableLambda(handle_tool_error)], exception_key="error"
    )


def _print_event(event: dict, _printed: set, max_length=1500):
    current_state = event.get("dialog_state")
    if current_state:
        print("Currently in: ", current_state[-1])
    message = event.get("messages")
    if message:
        if isinstance(message, list):
            message = message[-1]
        if message.id not in _printed:
            msg_repr = message.pretty_repr(html=True)
            if len(msg_repr) > max_length:
                msg_repr = msg_repr[:max_length] + " ... (truncated)"
            if "Tool Message" in msg_repr or "Tool Calls:" in msg_repr:
                pass
            else:
                print("INFO", msg_repr)
            _printed.add(message.id)

In [22]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnableConfig
from datetime import datetime
from textwrap import dedent

from langchain_core.tools import tool

@tool
def verify_customer_in_service_range(query: str) -> str:
    """Verify customer is in service range using customer Map"""
    response = in_service_range
    print("Calling verify_customer_in_service_range with query:", query, "Return:", response)
    return response

@tool
def log_customer_info(name_address_phone_number_email_address: str) -> str:
    """Log basic customer info to Talkdesk Contact"""
    print(f"Logging customer info: {name_address_phone_number_email_address}")
    return True
    
@tool
def retrieve_from_service_map(service: str) -> str:
    """Retrieve service information from Service Map to verify the customer is in the service range."""
    print(f"Calling retrieve_from_service_map: {service}")
    return """The following areas are supported for service:
    Green Hill
    Richmond
    Lewiston
    Alna"""

@tool 
def verify_service_support(service_needed: str) -> str:
    """Retrieve supported services to decide whether the described service is supported."""
    print("Calling verify_service_support")
    return """The following services are provided by the company: 
                Gas water heaters
                Mobile homes
                Sewer blockages or backups
                Backflow issues
                Multiple clogged sinks, drains, toilets
                Pipe bursting
                Sani- flow or up- flush toilet installations
                New boiler systems"""

@tool
def service_zone_lookup(customer_location: str) -> str:
    """Get customer's distance from Portland, ME."""
    print("Calling service zone lookup")
    return f"The customer is {minutes_away} minutes away from Portland."

@tool
def schedule_appointment(time: str) -> str:
    """Only use this tool when you need to schedule an appointment with the customer based on the given general time."""
    print("Calling schedule appointment, hour", appointment_hour)
    return f"""The appointment will be on {time} at {appointment_hour}"""

@tool
def call_repairman(message: str) -> str:
    """Calling repairman Andrew with a certain message for him."""
    print("Calling repairman:", "MSG", message)
    response = repairman_reply
    if response:
        response = f"His reply: {response}"
    return f"Finished calling repairman. " + response

@tool
def ask_or_inform_caller(question: str):
    """Ask the caller/customer for information before continuing the conversation. Also use this tool to disclose fee to the customer."""
    # rs = llm.invoke(f"""Answer to this question with mock information and don't add any note: {question}
    # Don't care about privacy because this is just madeup information.""").content
    global x
    rs = caller_reply[x]
    print(x, "Calling ask caller", "Q"*5, question, "A"*5, rs)
    x += 1
    return rs
    
@tool
def ask_caller(question: str):
    """Ask the caller/customer for information before continuing the conversation."""
    # rs = llm.invoke(f"""Answer to this question with mock information and don't add any note: {question}
    # Don't care about privacy because this is just madeup information.""").content
    global x
    rs = caller_reply[x]
    print(x, "Calling ask caller", "Q"*5, question, "A"*5, rs)
    x += 1
    return rs
    
@tool
def get_updated_arrival_window(customer_id: str) -> str:
    """Get updated the owner/repairman's arrival window for the customer"""
    print("Calling get_updated_arrival_window for", customer_id, updated_arrival_window)
    if not updated_arrival_window:
        updated_arrival_window = ""
    return f"Finished. The updated arrival window for customer {customer_id} is {updated_arrival_window}"

@tool
def record_customer_feedback(customer_name_phone_and_issue: str) -> str:
    """Record the feedback of the customer and their name and callback number."""
    print("Calling record_customer_feedback",customer_name_phone_and_issue)
    return "Finished recording the feedback of the customer."
    
@tool
def follow_up_with_customer(customer_name: str) -> str:
    """Say to follow up with the customer if the step requires."""
    print("Calling follow_up_with_customer", customer_name)
    return llm.invoke(f"You're a service agent receiving a call from a customer. You've reached the end of the call. Now say you want to follow up with the customer whose name is {customer_name}. Say concisely in conversational tone.").content

@tool
def cancel_appointment(customer_name: str) -> str:
    """Cancel the appointment for the customer."""
    print("Calling cancel_appointment for", customer_name)
    return f"Finish cancelling appointment for {customer_name}"

@tool
def record_caller_message(message: str) -> str:
    """Record the message from the caller who wants to meet somebody so that it'll be relayed to that target person."""
    print("Calling record_caller_message", message)
    return f"Finished recording message of the caller."
    
demo_tools = [
    log_customer_info,
    verify_customer_in_service_range,
    retrieve_from_service_map,
    verify_service_support,
    service_zone_lookup,
    schedule_appointment,
    call_repairman,
    record_customer_feedback,
    # ask_or_inform_caller,
    ask_caller,
    cancel_appointment,
    follow_up_with_customer,
    record_caller_message,
]

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

primary_assistant_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            dedent("""
                   
**Context:**
Today is 2024 December 25.
You are an agent for AT Plumbing Service, handling inbound calls. Calls can be from various sources—personal callers, sales representatives, new leads, existing customers with upcoming appointments, 
customers who have had previous services completed, or those inquiring about future services.  
You're provided with tools that you can select to implement the steps below. Choose the best matching tool for a step.

Below is a detailed inbound call-handling workflow for a plumbing service. Please use it as a procedural guide whenever you respond to a scenario involving an incoming call. 
Your goal is to follow the steps carefully, and simulate the decision-making process as outlined below.
It's crucially important that you utilize one of the tools whenever a step requires it.
Everything starts with Step 0 and move to the next steps based on their references.
---

**Key Steps and Decision Points:**

**Step 0: Identify Caller Type**
    - If the caller is personal (not related to a plumbing request), go to Step 1: Personal Call. 
    - If the caller is selling something, go to Step 2: Salesperson. 
    - If the caller is inquiring about a new lead (needs plumbing services), go to Step 3.1: Lead.
    - If the caller is a past customer or customer with current booking, go to Step 4: Customer Service.

**Step 1: Personal Call**
    - Tell the caller you will pass along a message and collect: Name, Callback Number, Message
    - Follow up with customer, if needed

**Step 2: Salesperson**
    - Politely decline and insist you're not interested
    - End the call
                   
**Step 3: Lead**: The caller needs plumbing services.
    Step 3.1: Take these substeps in order
        1. Ask the customer back about their basic information if it's not available initially: Name, Address, Phone Number and Email Address.
        2. Next, log the information in Talkdesk Contact. 
        3. Then go to Step 3.2
    Step 3.2: Verify customer is in service range using customer Map. If customer is not in service range, go to Step 3.3, else go to Step 3.4              
    Step 3.3:
        - Log customer information in Talkdesk Contact
        - Let the customer know we are unable to service them
        - End the call
    Step 3.4:
        - Refer to Knowledge Base to capture relevant information and verify we can service
            - If we cannot service, go to Step 3.3, else go to step 3.5

    Step 3.5:
        - Confirm customer's distance from Portland, ME using Service Zone Lookup tool
            - If customer is more than 60 minutes away, go to Step 3.3
            - Otherwise, ask the customer whether he needs ESTIMATE booking or SERVICE CALL booking.
                - If customer needs Estimate booking, go to Step 3.6
                - If customer needs Service Call booking, go to Step 3.9               

    Step 3.6
        - Determine how far the customer is
            - If less than 45 mins, go to Step 3.7
            - If 45-60 mins, go to Step 3.8
                   
    Step 3.7:
        - Disclose to the customer a $90 dispatch fee CREDITABLE to project cost
        - Schedule next available estimate appointment
        
    Step 3.8
        - Disclose $139.50 dispatch fee to the customer
        - Schedule next available estimate appointment
                   
    Step 3.9:
        - Determine if service is an emergency. Ask one of these questions to know it: "Is your house currently flooding? Can you shut the water oﬀ/do you
know where the shutoﬀs are? How long has this been going on for?"
            - Based on the answer, if it's an emergency, go to Step 3.10
            - Based on the answer, if it's NOT an emergency, go to Step 3.11

    Step 3.10:
        - Disclose $285 emergency dispatch fee.
        - Ask the customer about the appointment time. Decide whether the time is after or during business hours.
          If the appointment time is after hours, go to Step 3.11. 
          If the appointment time is during business hours, go to Step 3.12. 
          If the appointment is just a regular appointment booking, go to Step 3.13

    Step 3.11:
        - Schedule appointment for first appointment the NEXT MORNING. Tell the customer about the appointment time.
        - Call Andrew if true emergency (rare)
        - Commit to following up with customer                   
    
    Step 3.12:
        - Schedule first available appointment
        - Contact Andrew to see if he can service the emergency sooner
        - Follow up with customer, if needed
                   
    Step 3.13:
        - Disclose $139 Dispatch Fee to the customer
        - Book customer for next available appointment in ServiceTitan
        - End call/Follow up

**Step 4: Customer Service** (Past customer or customer with current booking)
    - If the customer calls about upcoming booking, go to step 4.1
    - If the customer calls about past/completed booking, go to step 4.2
    Step 4.1: Determine what the customer needs help with:
        - If Customer wants to change an upcoming appointment, go to step 4.3
        - If Customer wants an updated arrival window, go to step 4.4
        - If Customer wants to cancel upcoming appointment, go to step 4.5
    Step 4.2: Determine the reason for their call.
        - If it's urgent or customer is upset,
            1. Ask the customer Name, callback number. Then ask what issue they had. Next reach out to Andrew with the issue via SMS at any time of the day;
            2. Complete any required follow-up with the customer.
        - If the customer has a non-urgent question about their service, 1) Record the customer's name, contact info, and question and put into the Summary on Service Titan.
        - If the customer wants to book another appointment, go to Step 3.13
    Step 4.3: Ask the customer when the appointment is. If the appointment is TODAY, change the appointment and send an SMS message to the owner immediately to inform them of the same day scheduling changes. If the appointment is NOT TODAY, go to Step 3.13 above
    Step 4.4: 
        - Ask the customer name and email
        - Get updated arrival window for the customer.
        - If the updated arrival window is during/after the quoted window, then 
            - Tell customer you will try to contact the plumber; 
            - Call Andrew asking about his arrival; 
            - Follow up with customer as needed
        - If the updated arrival window is before the quoted window, inform the customer that their current arrival window is still the most accurate information. The plumber will contact when he is on the way.
    Step 4.5: Take the following actions
        1) Cancel the appointment 
        2) Ask customer why they are cancelling and record in call notes 
        3) If same day, send SMS to Andrew.
    Step 3.13: Disclose $139 Dispatch Fee to the customer. Book customer for next available appointment in ServiceTitan.
    

---

**Instructions**: Here are crucially important rules you should follow. You can do it.
- Given any inbound call scenario, emulate this decision tree logically and step-by-step.  
- Ensure you verify caller type, service area, urgency, and schedule availability.  
- Present all relevant options, gather the necessary info, and provide clear next steps, just as a real call agent following the company’s workflow would. This is crucial to my career.
- Gather the necessary info required by the current step before moving to the next step.
- Identify the next step by following the step reference that the current step points to.
- Take all the steps until there's no more step referenced.
- Only use a tool when the step needs it, otherwise, don't use any tool.
- Always be explicit about the fees the customer will have to pay at the end.
            """).strip()
        ),
        ("placeholder", "{messages}"),
    ]
).partial(time=datetime.now)


agent_prompt = PromptTemplate(
            template=dedent("""
                Answer the following questions as best you can. Choose the appropriate tools from this list to do that:
                {tools}

                Use the following ReAct template when reasoning. You run in a loop of Thought, Action, Action Input and Observation. Don't create Final Answer inside the loop.
                Only at the end of the loop you'll output Final Answer.
                Give the Final Answer only when you reach the last step of the calling scenario described in the Rule section. This is crucially important.
                
                ### The ReAct template:
                Question: the input question you must answer
                Thought: you should ALWAYS plan what to do.
                Action: the action to take, should be one of {tool_names}. The action must be the name of the tool only. Always show the next Step you'll take in the scenario workflow.
                Action Input: the input to the action, must not be empty
                Observation: the result of the action
                ... (this Thought/Action/Action Input/Observation can repeat N times.)

                Thought: I now know the final answer
                Final Answer: the final answer to the original input question
                
                ### Rigorously adhere to the following rules: 
                {primary_assistant_prompt}
                Continue until you reach the FINAL STEP of the scenario.
                You must use Thougth first before Action.
                </end of rules>
                
                Now, begin!

                Question: {question}
                Thought:
                {agent_scratchpad}
                """).strip(),
            input_variables=["tools", "tool_names", "question", "primary_assistant_prompt"]
        )

### SETTING UP PARAMETER VALUES
in_service_range = True
minutes_away = 60
appointment_hour = "9:15 AM"
# in_service_range = False
# minutes_away = 20
# appointment_hour = "9:15 PM"
# updated_arrival_window = "after quoted window"
agent = create_react_agent(llm, demo_tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=demo_tools, max_iterations=10, verbose=True)

# q = "Hi there, I need a plumber to fix my sink"
# q = "Hello. Can I speak to Mr. Hall please?"
# q = "Hi.  I want to ask about a complete booking."
# ag = agent_executor.invoke({"question": q,
#                            "primary_assistant_prompt": primary_assistant_prompt
#                            })


## PERSONAL CALL 

In [11]:
in_service_range = True
minutes_away = 60
repairman_reply = "" 
appointment_hour = "" 
updated_arrival_window = ""
x = 0
q = "Hello there. Can I speak to Mr. Hall please?"
caller_reply = ["I'm Harry. Phone is: 08545090889", "I want to tell him the contractor has agreed to undertake the project."]
ag = agent_executor.invoke({"question": q,
                           "primary_assistant_prompt": primary_assistant_prompt
                           })



> Entering new AgentExecutor chain...
I need to determine if the caller is making a personal call or if they are related to a plumbing request. Since the caller is asking for Mr. Hall, it suggests they may be leaving a message or seeking to speak with someone specific. I will treat this as a personal call and proceed to collect the necessary information.

Action: ask_caller  
Action Input: "Could you please provide your name, callback number, and the message you'd like me to pass along to Mr. Hall?"  0 Calling ask caller QQQQQ Could you please provide your name, callback number, and the message you'd like me to pass along to Mr. Hall? AAAAA I'm Harry. Phone is: 08545090889
I'm Harry. Phone is: 08545090889I have gathered the caller's name and phone number. Now, I need to ask for the message that Harry would like to pass along to Mr. Hall.

Action: ask_caller  
Action Input: "Thank you, Harry. What message would you like me to pass along to Mr. Hall?"  1 Calling ask caller QQQQQ Thank 

## <span style="color: red;">SALESPERSON</span>
Not robust to step: using tool when not needed; OR incorrect input tool

In [12]:
in_service_range = True
minutes_away = 60
repairman_reply = "" 
appointment_hour = "" 
updated_arrival_window = ""
x = 0
q = "Hello there. I have a new shampoo to offer. Could you spend sometime checking it out?"
caller_reply = ["I'm Harry. Phone is: 08545090889"]
ag = agent_executor.invoke({"question": q,
                           "primary_assistant_prompt": primary_assistant_prompt
                           })



> Entering new AgentExecutor chain...
Action: ask_caller  
Action Input: "I appreciate your offer, but I'm not interested in new products at this time. Is there anything else I can assist you with?"  0 Calling ask caller QQQQQ I appreciate your offer, but I'm not interested in new products at this time. Is there anything else I can assist you with? AAAAA I'm Harry. Phone is: 08545090889
I'm Harry. Phone is: 08545090889I need to log the customer's basic information now that I have their name and phone number.  
Action: log_customer_info  
Action Input: "Harry, 08545090889"  Logging customer info: Harry, 08545090889
TrueI have successfully logged the customer's basic information. Now, I need to follow up with the customer to ensure they are aware of the next steps.  
Action: follow_up_with_customer  
Action Input: "Thank you, Harry. I've logged your information. If you have any plumbing needs in the future, feel free to reach out!"  Calling follow_up_with_customer Thank you, Harry. I'v

## LEAD, NOT IN SERVICE RANGE

In [11]:
in_service_range = False
minutes_away = 60
repairman_reply = "" 
appointment_hour = "" 
updated_arrival_window = ""
x = 0
q = "Hi there, I need a plumber to fix my sink"
caller_reply = ["Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com"]
ag = agent_executor.invoke({"question": q,
                           "primary_assistant_prompt": primary_assistant_prompt
                           })



> Entering new AgentExecutor chain...
I need to identify the caller type to determine the next steps. Since the caller needs plumbing services, I will proceed to Step 3.1, which involves gathering basic information from the customer.

Action: ask_caller  
Action Input: "Could you please provide me with your name, address, phone number, and email address?"  Calling ask caller QQQQQ Could you please provide me with your name, address, phone number, and email address? AAAAA Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com
Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.comI have gathered the customer's basic information: name (Steven), address (209 Palm Street), phone number (118924568790), and email address (steve_james12@gmail.com). Now, I will log this information in Talkdesk Contact.

Action: log_customer_info  
Action Input: "Steven, 209 Palm Street, 118924568790, steve_james12@gmail.com"  Logging customer i

## LEAD, IN SERVICE, MORE THAN 60 MINS AWAY

In [24]:
in_service_range = True
minutes_away = 76
repairman_reply = "" 
appointment_hour = "" 
updated_arrival_window = ""
x = 0
q = "Hi there, I need a plumber to fix my sink"
caller_reply = ["Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com", "Oh. THANKS"]
ag = agent_executor.invoke({"question": q,
                           "primary_assistant_prompt": primary_assistant_prompt
                           })



> Entering new AgentExecutor chain...
I need to identify the caller type to determine the appropriate next steps. Since the caller is inquiring about plumbing services, I will proceed to Step 3.1, which involves gathering basic information from the customer. 

Action: ask_caller  
Action Input: "Could you please provide me with your name, address, phone number, and email address?"  0 Calling ask caller QQQQQ Could you please provide me with your name, address, phone number, and email address? AAAAA Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com
Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.comI have gathered the customer's basic information: name, address, phone number, and email address. Now, I will log this information into Talkdesk Contact as per Step 3.2.

Action: log_customer_info  
Action Input: "Steven, 209 Palm Street, 118924568790, steve_james12@gmail.com"  Logging customer info: Steven, 209 Palm 

## LEAD, IN SERVICE RANGE, ESTIMATE BOOKING, 45-60 MINS

In [32]:
in_service_range = True
minutes_away = 50
repairman_reply = "" 
appointment_hour = "" 
updated_arrival_window = ""
q = "Hi there, I need a plumber to fix my sink"
x = 0
caller_reply = ["Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com",
                   "estimate booking please",
                   "yes. I accept it."
               ]
ag = agent_executor.invoke({"question": q,
                           "primary_assistant_prompt": primary_assistant_prompt
                           })



> Entering new AgentExecutor chain...
I need to identify the caller type. Since the caller is requesting plumbing services, I will proceed to Step 3.1, which involves gathering their basic information: Name, Address, Phone Number, and Email Address.

Action: ask_caller  
Action Input: "Could you please provide me with your name, address, phone number, and email address?"  0 Calling ask caller QQQQQ Could you please provide me with your name, address, phone number, and email address? AAAAA Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com
Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.comI have gathered the customer's basic information: Name (Steven), Address (209 Palm Street), Phone Number (118924568790), and Email Address (steve_james12@gmail.com). Now, I will log this information in Talkdesk Contact as per Step 3.2.

Action: log_customer_info  
Action Input: "Steven, 209 Palm Street, 118924568790, steve_jame

## LEAD, IN SERVICE RANGE, ESTIMATE BOOKING, LESS THAN 45 MINS

In [20]:
in_service_range = True
minutes_away = 30
repairman_reply = "" 
appointment_hour = "" 
updated_arrival_window = ""
q = "Hi there, I need a plumber to fix my sink"
x = 0
caller_reply = ["Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com",
                   "estimate booking please",
                   "yes. I accept it."
               ]
ag = agent_executor.invoke({"question": q,
                           "primary_assistant_prompt": primary_assistant_prompt
                           })



> Entering new AgentExecutor chain...
I need to determine if the caller is a new lead inquiring about plumbing services. Since they are requesting a plumber, I will proceed to ask for their basic information: Name, Address, Phone Number, and Email Address.

Action: ask_or_inform_caller  
Action Input: "Could you please provide me with your name, address, phone number, and email address?"  0 Calling ask caller QQQQQ Could you please provide me with your name, address, phone number, and email address? AAAAA Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com
Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.comI have gathered the customer's basic information: Name (Steven), Address (209 Palm Street), Phone Number (118924568790), and Email Address (steve_james12@gmail.com). Now, I will log this information in Talkdesk Contact.

Action: log_customer_info  
Action Input: "Steven, 209 Palm Street, 118924568790, steve_jam

## <span style="color: red;">LEAD, IN SERVICE RANGE, SERVICE CALL BOOKING, EMERGENCY, AFTER HOURS</span>: NO FEE DISCLOSED

In [23]:
in_service_range = True
minutes_away = 30
repairman_reply = "" 
appointment_hour = "" 
updated_arrival_window = ""
q = "Hi there, I need a plumber to fix my sink"
x = 0
caller_reply = ["Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com",
                   "service call booking please",
                   "yes. It's now flooding.",
                "after hours please"
               ]
ag = agent_executor.invoke({"question": q,
                           "primary_assistant_prompt": primary_assistant_prompt
                           })



> Entering new AgentExecutor chain...
I need to identify the caller type to determine the next steps. Since the caller is inquiring about plumbing services, I will proceed to Step 3.1, which involves asking for their basic information: Name, Address, Phone Number, and Email Address.

Action: ask_caller  
Action Input: "Could you please provide me with your name, address, phone number, and email address?"  0 Calling ask caller QQQQQ Could you please provide me with your name, address, phone number, and email address? AAAAA Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com
Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.comI have gathered the customer's basic information: Name (Steven), Address (209 Palm Street), Phone Number (118924568790), and Email Address (steve_james12@gmail.com). Now, I will log this information in Talkdesk Contact as per Step 3.2.

Action: log_customer_info  
Action Input: "Steven, 209 Pal

## LEAD, IN SERVICE RANGE, SERVICE CALL BOOKING, EMERGENCY, DURING HOURS

In [38]:
in_service_range = True
minutes_away = 30
repairman_reply = "" 
appointment_hour = "9:00 AM" 
updated_arrival_window = ""
q = "Hi there, I need a plumber to fix my sink"
x = 0
caller_reply = ["Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com",
                   "service call booking please",
                   "yes. It's now flooding.",
                "during hours please"
               ]
ag = agent_executor.invoke({"question": q,
                           "primary_assistant_prompt": primary_assistant_prompt
                           })



> Entering new AgentExecutor chain...
I need to determine if the caller is a new lead needing plumbing services. Since they are requesting a plumber, I will proceed to Step 3.1, which involves asking for their basic information: name, address, phone number, and email address.

Action: ask_caller  
Action Input: "Could you please provide me with your name, address, phone number, and email address?"  0 Calling ask caller QQQQQ Could you please provide me with your name, address, phone number, and email address? AAAAA Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com
Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.comNow that I have gathered the customer's basic information, I will log this information into Talkdesk Contact to ensure it is recorded properly.

Action: log_customer_info  
Action Input: "Steven, 209 Palm Street, 118924568790, steve_james12@gmail.com"  Logging customer info: Steven, 209 Palm Street, 1

## LEAD, SERVICE RANGE, SERVICE CALL, NOT EMERGENCY

In [27]:
in_service_range = True
minutes_away = 30
repairman_reply = "" 
appointment_hour = "9:00 AM" 
updated_arrival_window = ""
q = "Hi there, I need a plumber to fix my sink"
x = 0
caller_reply = ["Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com",
                   "service call booking please",
                   "No. It's not flooding.",
                    "No. Thanks"
               ]
ag = agent_executor.invoke({"question": q,
                           "primary_assistant_prompt": primary_assistant_prompt
                           })



> Entering new AgentExecutor chain...
I need to determine if the caller is a new lead looking for plumbing services. Since they mentioned needing a plumber, I will proceed to ask for their basic information: name, address, phone number, and email address.

Action: ask_caller  
Action Input: "Could you please provide me with your name, address, phone number, and email address?"  0 Calling ask caller QQQQQ Could you please provide me with your name, address, phone number, and email address? AAAAA Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com
Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.comNow that I have gathered the customer's basic information, I will log this information into Talkdesk Contact to ensure it is recorded properly.

Action: log_customer_info  
Action Input: "Steven, 209 Palm Street, 118924568790, steve_james12@gmail.com"  Logging customer info: Steven, 209 Palm Street, 118924568790, steve_ja

## LEAD, IN SERVICE RANGE, SERVICE CALL, EMERGENCY, REGULAR APPOINTMENT

In [15]:
in_service_range = True
minutes_away = 30
repairman_reply = "" 
appointment_hour = "9:00 AM" 
updated_arrival_window = ""
q = "Hi there, I need a plumber to fix my sink"
x = 0
caller_reply = ["Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com",
                   "service call booking please",
                   "Yes. It's now flooding.",
                "Just regular appointment please"
               ]
ag = agent_executor.invoke({"question": q,
                           "primary_assistant_prompt": primary_assistant_prompt
                           })



> Entering new AgentExecutor chain...
I need to determine if the caller is a new lead who needs plumbing services. Since they are requesting a plumber, I will proceed to ask for their basic information: Name, Address, Phone Number, and Email Address.

Action: ask_caller  
Action Input: "Could you please provide me with your Name, Address, Phone Number, and Email Address?"  0 Calling ask caller QQQQQ Could you please provide me with your Name, Address, Phone Number, and Email Address? AAAAA Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com
Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.comI have gathered the necessary basic information from the caller, Steven. Now, I will log this information into the Talkdesk Contact system.

Action: log_customer_info  
Action Input: "Steven, 209 Palm Street, 118924568790, steve_james12@gmail.com"  Logging customer info: Steven, 209 Palm Street, 118924568790, steve_james12@gma

## CUSTOMER SERVICE, UPCOMING BOOKING, CHANGE UNCOMING APPOINTMENT, APPOINTMENT TODAY

In [38]:
in_service_range = True
minutes_away = 30
repairman_reply = "" 
appointment_hour = "9:00 AM" 
updated_arrival_window = ""
q = "Hi there. I have booking. I'm calling about upcoming booking."
x = 0
caller_reply = [
                "Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com",
                "I want to change my upcoming appointment",
                "It's today."
               ]
ag = agent_executor.invoke({"question": q,
                           "primary_assistant_prompt": primary_assistant_prompt
                           })



> Entering new AgentExecutor chain...
Action: ask_caller  
Action Input: "Can I have your name and the phone number associated with your booking, please?"  0 Calling ask caller QQQQQ Can I have your name and the phone number associated with your booking, please? AAAAA Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com
Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.comI have gathered the customer's name, address, phone number, and email. Now, I need to log this information in Talkdesk Contact.  
Action: log_customer_info  
Action Input: "Steven, 209 Palm Street, 118924568790, steve_james12@gmail.com"  Logging customer info: Steven, 209 Palm Street, 118924568790, steve_james12@gmail.com
TrueI have successfully logged the customer's information. Now, I need to determine what the customer needs help with regarding their upcoming booking.  
Action: ask_caller  
Action Input: "What do you need help with regarding you

## CUSTOMER SERVICE, UPCOMING BOOKING, CHANGE UNCOMING APPOINTMENT, APPOINTMENT NOT TODAY

In [56]:
in_service_range = True
minutes_away = 30
repairman_reply = "" 
appointment_hour = "9:00 AM" 
updated_arrival_window = ""
q = "Hi there. I have booking. I'm calling about upcoming booking."
x = 0
caller_reply = [
                "Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com",
                "I want to change my upcoming appointment",
                "It's tomorrow, not today",
                "No thanks"
               ]
ag = agent_executor.invoke({"question": q,
                           "primary_assistant_prompt": primary_assistant_prompt
                           })



> Entering new AgentExecutor chain...
Action: ask_caller  
Action Input: "Could you please provide your name and the phone number associated with the booking?"  0 Calling ask caller QQQQQ Could you please provide your name and the phone number associated with the booking? AAAAA Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com
Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.comI have gathered the customer's name, address, phone number, and email address. Now I will log this information into the system.

Action: log_customer_info  
Action Input: "Steven, 209 Palm Street, 118924568790, steve_james12@gmail.com"  Logging customer info: Steven, 209 Palm Street, 118924568790, steve_james12@gmail.com
TrueThe customer information has been successfully logged. Now, I need to determine what the customer needs help with regarding their upcoming booking.

Action: ask_caller  
Action Input: "What specific assistance do you 

## CUSTOMER SERVICE, UPCOMING BOOKING, UPDATE ARRIVAL WINDOW, BEFORE THE QUOTED WINDOW

In [58]:
in_service_range = True
minutes_away = 30
repairman_reply = "" 
appointment_hour = "9:00 AM" 
updated_arrival_window = "before quoted window"
q = "Hi there. I have booking. I'm calling about upcoming booking."
x = 0
caller_reply = [
                "Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com",
                "I want to get updated on the arrival window.",
                "before quoted window",
                "No thanks"
               ]
ag = agent_executor.invoke({"question": q,
                           "primary_assistant_prompt": primary_assistant_prompt
                           })



> Entering new AgentExecutor chain...
Action: ask_caller  
Action Input: "Can I have your name and the phone number associated with the booking?"  0 Calling ask caller QQQQQ Can I have your name and the phone number associated with the booking? AAAAA Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com
Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.comI have gathered the customer's name, address, phone number, and email address. Now I will log this information into the system.  
Action: log_customer_info  
Action Input: "Steven, 209 Palm Street, 118924568790, steve_james12@gmail.com"  Logging customer info: Steven, 209 Palm Street, 118924568790, steve_james12@gmail.com
TrueThe customer information has been successfully logged. Now, I will determine what the customer needs help with regarding their upcoming booking.  
Action: ask_caller  
Action Input: "What do you need help with regarding your upcoming booking?" 

## CUSTOMER SERVICE, UPCOMING BOOKING, CANCEL APPOINTMENT

In [65]:
in_service_range = True
minutes_away = 30
repairman_reply = "" 
appointment_hour = "9:00 AM" 
updated_arrival_window = "before quoted window"
q = "Hi there. I have booking. I'm calling about upcoming booking."
x = 0
caller_reply = [
                "Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com",
                "I want to cancel my upcoming appointment.",
                "I have an unexpected situation that I have to leave my house.",
                "Yes, please.",
                "No problem."
               ]
ag = agent_executor.invoke({"question": q,
                           "primary_assistant_prompt": primary_assistant_prompt
                           })



> Entering new AgentExecutor chain...
Action: ask_caller  
Action Input: "Can I have your name and callback number, please?"  0 Calling ask caller QQQQQ Can I have your name and callback number, please? AAAAA Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com
Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.comI have gathered the caller's name, address, phone number, and email. Now, I will log this customer information into Talkdesk Contact.  
Action: log_customer_info  
Action Input: "Steven, 209 Palm Street, 118924568790, steve_james12@gmail.com"  Logging customer info: Steven, 209 Palm Street, 118924568790, steve_james12@gmail.com
TrueI have successfully logged the customer's information. Now, I will determine what the customer needs help with regarding their upcoming booking.  
Action: ask_caller  
Action Input: "What do you need help with regarding your upcoming booking?"  1 Calling ask caller QQQQQ What do 

## CUSTOMER SERIVICE, COMPLETED BOOKING, URGENT/UPSET

In [71]:
in_service_range = True
minutes_away = 30
repairman_reply = "" 
appointment_hour = "9:00 AM" 
updated_arrival_window = "before quoted window"
q = "Hi there. I have booking. I'm calling about my past booking."
x = 0
caller_reply = [
                "Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com",
                "I want to comment on my past appointment. I'm very dissatisfied about it.",
               ]
ag = agent_executor.invoke({"question": q,
                           "primary_assistant_prompt": primary_assistant_prompt
                           })



> Entering new AgentExecutor chain...
I need to determine the reason for the customer's call regarding their past booking. Since they mentioned they have a booking, I will proceed to Step 4.2 to identify the issue they are experiencing.

Action: ask_caller  
Action Input: "Can you please tell me the reason for your call regarding your past booking?"  0 Calling ask caller QQQQQ Can you please tell me the reason for your call regarding your past booking? AAAAA Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com
Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.comI have gathered the customer's name, address, phone number, and email. Now, I will proceed to determine if the customer has an urgent issue or if it's a non-urgent question about their service. Since the customer provided their details, I will ask if they have an urgent issue.

Action: ask_caller  
Action Input: "Is your issue urgent or are you calling with 

## CUSTOMER SERIVICE, COMPLETED BOOKING, NON-URGENT QUESTION

In [75]:
in_service_range = True
minutes_away = 30
repairman_reply = "" 
appointment_hour = "9:00 AM" 
updated_arrival_window = "before quoted window"
q = "Hi there. I have booking. I'm calling about my past booking."
x = 0
caller_reply = ["I want to ask more about my completed booking.",
                "Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com",
                "Did the repairman left something at my house. I saw a strange spanner here."
               ]
ag = agent_executor.invoke({"question": q,
                           "primary_assistant_prompt": primary_assistant_prompt
                           })



> Entering new AgentExecutor chain...
I need to determine the reason for the customer's call regarding their past booking. This will help me decide the next steps to take. 

Action: ask_caller  
Action Input: "Can you please tell me the reason for your call regarding your past booking?"  0 Calling ask caller QQQQQ Can you please tell me the reason for your call regarding your past booking? AAAAA I want to ask more about my completed booking.
I want to ask more about my completed booking.I need to gather more specific information about the customer's completed booking to assist them effectively. 

Action: ask_caller  
Action Input: "Could you please provide me with your name and the details of the issue you had with your completed booking?"  1 Calling ask caller QQQQQ Could you please provide me with your name and the details of the issue you had with your completed booking? AAAAA Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com
Steven. 209 Palm 

## <span style="color: red;">CUSTOMER SERIVICE, COMPLETED BOOKING, BOOK ANOTHER APPOINTMENT</span>: NO FEE DISCLOSED

In [24]:
in_service_range = True
minutes_away = 30
repairman_reply = "" 
appointment_hour = "9:00 AM" 
updated_arrival_window = "before quoted window"
q = "Hi there. I have booking. I'm calling about my past booking."
x = 0
caller_reply = ["I want to book a new appointment.",
                "Steven. 209 Palm Street. my phone is 118924568790. and email is steve_james12@gmail.com",
               ]
ag = agent_executor.invoke({"question": q,
                           "primary_assistant_prompt": primary_assistant_prompt
                           })



> Entering new AgentExecutor chain...
I need to determine the reason for the customer's call regarding their past booking. Since they mentioned they have a booking, I will proceed to Step 4.2 to find out more about their inquiry.

Action: ask_caller  
Action Input: "Could you please tell me the reason for your call regarding your past booking?"  0 Calling ask caller QQQQQ Could you please tell me the reason for your call regarding your past booking? AAAAA I want to book a new appointment.
I want to book a new appointment.The customer wants to book a new appointment. Since they are a past customer, I will now proceed to Step 3.13, which involves disclosing the dispatch fee and booking them for the next available appointment.

Action: record_customer_feedback  
Action Input: "I will need your name, phone number, and the details of the service you would like to book."  Calling record_customer_feedback I will need your name, phone number, and the details of the service you would like to 